<a href="https://colab.research.google.com/github/debu-sinha/building-ai-agents/blob/main/Product_Q_A_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Install all dependency packages
#Remember to execute this before running any of the exercises

!pip install langchain
!pip install langchain-openai
!pip install langchain_community
!pip install langchain_chroma
!pip install langgraph

!pip install tenacity
!pip install pysqlite3-binary
!pip install pandas
!pip install pypdf
!pip install nbformat

### Setup Models

In [7]:
from google.colab import userdata

from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage

from langchain_openai import OpenAIEmbeddings, ChatOpenAI

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=userdata.get('oai'))
chat_model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=userdata.get("oai"))

In [8]:
import pandas as pd

product_pricing_df = pd.read_csv("./data/Laptop pricing.csv")
product_pricing_df.head()

,Name,Price,ShippingDays
0,AlphaBook Pro,1499,2
1,GammaAir X,1399,7
2,SpectraBook S,2499,7
3,OmegaPro G17,2199,14
4,NanoEdge Flex,1699,2


### Add Product Pricing function tool

In [9]:
@tool
def get_laptop_price(laptop_name: str) -> int:
  """
  This function return the price of a laptop given its name.
  It performs a substring match between input name and the name of the laptop.
  If a match is found it returns the price of the laptop.
  If no match is found it returns -1.
  """
  match_records_df = product_pricing_df[product_pricing_df["Name"].str.contains("^" + laptop_name, case=False)]
  if len(match_records_df) > 0:
    return match_records_df["Price"].iloc[0]
  else:
    return -1

In [10]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Add Product Features Retrieval Tool

In [11]:
from langchain_chroma import Chroma
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

loader = PyPDFLoader("./data/Laptop product descriptions.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

prod_feature_store = Chroma.from_documents(splits, embedding_model)

get_product_features = create_retriever_tool(prod_feature_store.as_retriever(search_kwargs={"k": 5}),
                                             name="Get_Product_Features",
                                             description="""This store contains details about Laptops. It lists the available laptops and theor features
                                             including CPU, memory, storage, design and advantages
                                             """
                                             )

### Setup a Product QnA chatbot

In [18]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

tools = [get_product_features, get_laptop_price]

checkpointer = MemorySaver()

product_qna_agent = create_react_agent(model=chat_model,
                                       tools=tools,
                                       prompt=system_prompt,
                                       checkpointer=checkpointer,
                                       debug=False)

### To maintain memory, each request should be in the context of a thread.
### Each user conversation will use a separate thread ID

In [19]:
import uuid

config = {"configurable": {"thread_id": uuid.uuid4()}}

inputs = {"messages": [("user", "Give me a list of available laptop names and their features")]}

for stream in product_qna_agent.stream(inputs, config=config, stream_mode="values"):
  message = stream["messages"][-1]
  if isinstance(message, tuple):
    print(message)
  else:
    message.pretty_print()

================================ Human Message =================================

Give me a list of available laptop names and their features
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_uOCdIQyWjarr6dia2DwQTUae)
 Call ID: call_uOCdIQyWjarr6dia2DwQTUae
  Args:
    query:
================================= Tool Message =================================
Name: Get_Product_Features

Fictional Laptop Descriptions
AlphaBook Pro
The AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,
and a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of
power and portability.
GammaAir X
GammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB
NVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a
portable design.
SpectraBook S
Designed for power users, SpectraBook S features an Intel Core

### Execute the Product QnA Chatbot

In [20]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_qna_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")

----------------------------------------
USER : Hello
AGENT : Hello! How can I assist you today?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : That's great! What specific features or requirements are you looking for in a laptop? This could include things like the processor, memory, storage, or any other preferences you have.
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Here are some available laptops:

1. **AlphaBook Pro** - A sleek ultrabook with a 12th Gen Intel i7 processor, 16GB RAM, and a 1TB SSD.
2. **GammaAir X** - Features an AMD Ryzen 7 processor, 32GB RAM, and a 512GB NVMe SSD, known for its portability.
3. **SpectraBook S** - Designed for power users with an Intel Core i9 processor, 64GB RAM, and a 2TB SSD, ideal for intensive tasks.
4. **OmegaPro G17** - A gaming laptop with a Ryzen 9 5900HX CPU, 32GB RAM, and a 1TB SSD, featuring a high refresh rate 17-inch display.
5. **NanoEdge Fle

### Example of tracking multiple conversations.

In [21]:
#Create a new thread
config_user1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_user2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

def track_conversation(user, message, config):
  print(f"----------------------------------------\n{user} : {message}")
  user_message = {"messages":[HumanMessage(message)]}
  ai_response = product_qna_agent.invoke(user_message, config=config)
  print(f"AGENT : {ai_response['messages'][-1].content}")

track_conversation("User1", "Hello", config_user1)
track_conversation("User2", "I am looking to buy a laptop", config_user2)
track_conversation("User1", "Give me a list of available laptop names", config_user1)
track_conversation("User2", "Tell me about the features of  SpectraBook", config_user2)
track_conversation("User1", "How much does it cost?", config_user1)

----------------------------------------
User1 : Hello
AGENT : Hello! How can I assist you today?
----------------------------------------
User2 : I am looking to buy a laptop
AGENT : That's great! I can help you with that. What specific features or specifications are you looking for in a laptop? For example, do you have a preferred brand, size, or purpose such as gaming, business, or general use?
----------------------------------------
User1 : Give me a list of available laptop names
AGENT : I currently don't have access to the list of available laptop names directly. However, if you have a specific laptop in mind or a type of laptop you're interested in, I can help you find information about it!
----------------------------------------
User2 : Tell me about the features of  SpectraBook
AGENT : The SpectraBook S is designed for power users and comes with impressive features. Here are the key specifications:

- **Processor:** Intel Core i9
- **Memory:** 64GB RAM
- **Storage:** 2TB SSD